<a href="https://colab.research.google.com/github/tkeshun/til/blob/master/CNN(FuncAPI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash 
echo "[CPU]"; ls cpu
echo; echo "[Memory]"; free -h
echo; echo "[Disk]"; df -h /
echo; echo "[GPU]" ; nvidia-smi


[CPU]

[Memory]
              total        used        free      shared  buff/cache   available
Mem:            12G        560M         10G        900K        2.0G         11G
Swap:            0B          0B          0B

[Disk]
Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   32G   34G  49% /

[GPU]
Tue Mar  3 14:55:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+---

ls: cannot access 'cpu': No such file or directory


In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [11]:
%matplotlib
import numpy as np
from PIL import Image #画像処理ライブラリPillowのImageクラスをインポート
import matplotlib.pyplot as plt
import os 
import sys
from tensorflow import keras
from tensorflow.keras.datasets import mnist #kerasに内蔵されたMNISTを使う

Using matplotlib backend: agg


In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D

#dataのロード
(x_train, y_train) , (x_test,y_test) = mnist.load_data()
#detaの1次元化
num_train_data = x_train.shape[0]
num_test_data  = x_test.shape[0]
num_flatten_data = x_train.shape[1] * x_train.shape[2]
#x_train = x_train.reshape(num_train_data,num_flatten_data)
#x_test  = x_test.reshape(num_test_data, num_flatten_data)

#0~1の範囲に収まるようにする
num_gray_scale_max = 255
x_train = x_train.astype('float32') / num_gray_scale_max
x_test  = x_test.astype('float32')  / num_gray_scale_max
 
#one-hot 第一パラメータに指定した変数を第二パラメータで指定した次元数でone-hotベクトルで変換できる
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)
#モデル定義(Functionl)
"""
Functional APIでは呼び出す関数やクラスの末尾にテンソルをつけ，
層の演算を適用して結果としてテンソル変数を返す
"""
input_shape = (28,28)
#この下２つを使うときはデータをreshapeで一列にしない
inputs = Input(shape=input_shape)#最初は28*28じゃダメ,エラー：Input 0 of layer conv2d_1 is incompatible with the layerが出る
hidden = Reshape((28,28,1), input_shape = (28,28))(inputs)

x = Conv2D(32,kernel_size=(3,3), activation='relu') (hidden)
x = Conv2D(64,(3,3),activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2)) (x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=[inputs], outputs=[outputs])
model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0   

In [20]:
#訓練開始
batch_size = 128
epochs = 1000
history = model.fit(x_train, y_train,#訓練データと正解データ
                    batch_size = batch_size, #バッチサイズ
                    epochs=epochs,#エポック数
                    verbose=1,#訓練の進行度合いを表示
                    validation_data=(x_test, y_test)) #テスト

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 14s 240us/sample - loss: 2.2625 - accuracy: 0.1535 - val_loss: 2.1947 - val_accuracy: 0.3218
Epoch 2/1000
60000/60000 [==============================] - 8s 129us/sample - loss: 2.1649 - accuracy: 0.2744 - val_loss: 2.0780 - val_accuracy: 0.5471
Epoch 3/1000
60000/60000 [==============================] - 8s 129us/sample - loss: 2.0473 - accuracy: 0.3843 - val_loss: 1.9273 - val_accuracy: 0.6582
Epoch 4/1000
60000/60000 [==============================] - 8s 129us/sample - loss: 1.8951 - accuracy: 0.4757 - val_loss: 1.7388 - val_accuracy: 0.7216
Epoch 5/1000
60000/60000 [==============================] - 8s 129us/sample - loss: 1.7202 - accuracy: 0.5468 - val_loss: 1.5273 - val_accuracy: 0.7611
Epoch 6/1000
60000/60000 [==============================] - 8s 129us/sample - loss: 1.5380 - accuracy: 0.5993 - val_loss: 1.3154 - val_accuracy: 0.7869
Epoch 7/1000
60000/60000 [===========

KeyboardInterrupt: ignored

In [21]:
#訓練時の精度と損失の描画
#フォントの制御
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.patheffects as path_effects


#精度グラフの描画
plt.figure(figsize=(14,10))
plt.plot(history['acc'],
         color='b',
         linestyle='-',
         linewidth=3,
         path_effects=[path_effects.SimpleLineShadow(),
                       path_effects.Normal()])
plt.plot(history.history['val_acc'],
         color='r',
         linestyle='--',
         linewidth=3,
         path_effects=[path_effects.SimpleLineShadow(),
                       path_effects.Normal()])
plt.tick_params(labelsize=18)

plt.title('エポック-精度グラフ(MNIST_CNN)',fontsize=30,font_properties=fp2)
plt.ylabel('制度')
plt.xlabel('エポック')
plt.legend(['訓練','テスト'], loc='best', fontsize=25, prop=fp2)

plt.show()

NameError: ignored